<a href="https://colab.research.google.com/github/OrestisPnk/Movie-Recommendation-System/blob/master/reccomender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [ ]:
from google.colab import files
uploaded = files.upload()   

Saving u.data to u.data


In [ ]:
uploaded2 = files.upload()   

Saving ua.base to ua.base


In [ ]:
uploaded3 = files.upload()   

Saving ua.test to ua.test


In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from itertools import zip_longest
import random
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [ ]:
# load data, set new as our original dataframe 
new = pd.read_csv('u.data', sep='\t',names= ['userID','movieID','rating','timestamp'])
new = new.pivot(index='userID',columns='movieID',values='rating')

In [ ]:
# load training data, set new2 as our original training dataframe 
new2 = pd.read_csv('ua.base', sep='\t',names= ['userID','movieID','rating','timestamp'])
new2 = new2.pivot(index='userID',columns='movieID',values='rating')

In [ ]:
# load training data, set new3 as our testing dataframe 
new3 = pd.read_csv('ua.test', sep='\t',names= ['userID','movieID','rating','timestamp'])
new3 = new3.pivot(index='userID',columns='movieID',values='rating')

In [ ]:
#  Η συνάρτηση που υπολογίζει τη συσχέτιση Pearson μεταξύ των χρηστών του dataset και του χρήστη που μας ενδιαφέρει

def correlate(u1,new):
# initialize pearson's numerator and denominator
      num,std1,std2=0,0,0
      Plist=[]
   # Συλλέγω όλες τις αξιολογήσεις του χρήστη και αφαιρώ τη μέση τιμή αυτών
      u1=u1.dropna()-(u1.mean())
      
      for n in range(0,len(new)):
        us1,us2=[],[]
        
      # Συλλέγω όλες τις αξιολογήσεις του υποψήφιου γείτονα και αφαιρώ τη μέση τιμή αυτών
        u2=new.iloc[n]
        u2=u2.dropna()-(u2.mean())

      # indexes of movies that both users have rated
        ind=u1.index.intersection(u2.index)

        for i in range(0,len(ind)):

          us1.append(u1.loc[ind[i]])
          us2.append(u2.loc[ind[i]])

        if len(ind)>12:# and len(dif)>70 :

       # pearson's coefficient numerator, covariance
          num=sum([a*b for a,b in zip(us1,us2)])

       # denominator, product of standard deviations
          std1=np.sqrt(sum([a*b for a,b in zip(us1,us1)]))
          std2=np.sqrt(sum([a*b for a,b in zip(us2,us2)]))

       # compute pearson's coefficient , +1  to avoid negative values
          sim=(num/(std1*std2))+1

        else :sim=0

        Plist.append(sim)
      return Plist

# Διαμόρφωση της Λίστας με τους γείτονες
def presentList(Plist,i):
    PEARSON = pd.DataFrame({'correlation':Plist})
    a=False
    if i==1:
      u="userID"
      j=11
    else:
      u="chromosomeID"
      j=len(Plist)
      a=True
    PEARSON.insert(1,u, range(1, 1 + len(Plist)))
    PEARSON=PEARSON.sort_values(by='correlation', ascending=a)
    PEARSON=PEARSON.head(j)
    return PEARSON

# δείκτες-ids για τους 10 κοντινότερους γείτονες
def topIndex(PEARSON):
    top10=PEARSON.index.to_list()
    # top10 user's ID
    top10.pop(0)
    return top10  

# Εύρεση των ταινιών που έχουν αξιολογήσει μόνο οι γείτονες, (ΤαινίεςΓείτονα1)ΕΝΩΣΗ...ΕΝΩΣΗ(ΤαινίεςΓείτονα10)-{ΤαινίεςΧρηστη}
def findRecommendations(u,new,top10):
    u1=new.iloc[u].dropna()
    recom=u1.index
    for t in top10:
        u2=new.iloc[t].dropna()
        recom=recom.union(u2.index)   
    recom=recom.difference(u1.index)
    return recom

def findRecommendations2(new,new2,top10,u):
    u1=new2.iloc[u].dropna()
    recom=u1.index
    for t in top10:
        u2=new.iloc[t].dropna()
        recom=recom.union(u2.index)
    recom=recom.difference(u1.index)
    return recom

# Συνάρτηση από την οποία προκύπτουν το fitness των ατόμων του πληθυσμού, δηλαδή o μέσος όρος των συσχετίσεων Pearson του κάθε ατόμου με τους γείτονες
def fitness(pop,Neighbors,recom):     
    Fitlist,IndCorList=[],[]
    for i in range(0,len(pop)):
        individual=pop.iloc[i]
        #Individuals mean correlation
        IndCorList=correlate2(individual,Neighbors,recom)
        # IndCorList=correlate(individual,Neighbors)
        Fitlist.append(np.mean(IndCorList))
    return Fitlist

# Ίδια με την correlate(), αλλά την χρησιμοποιώ αποκλειστικά μέσα στη fitness, και παίρνει επιπλέον το όρισμα των ταινιών που έχει δει η γειτονιά αλλά όχι ο χρηστης
def correlate2(u1,new,recom):
      num,std1,std2=0,0,0
      Plist=[]
      u1=u1.dropna()-(u1.mean())
      
      for n in range(0,len(new)):
        us1,us2=[],[]
        
        u2=new.iloc[n]
        u2=u2.dropna()-(u2.mean())
        
 # indexes of movies that only neighborhood has seen
        ind=recom.intersection(u2.index)

        for i in range(0,len(ind)):

          us1.append(u1.loc[ind[i]])
          us2.append(u2.loc[ind[i]])

        if len(ind)>2:# and len(dif)>70 :

          num=sum([a*b for a,b in zip(us1,us2)])

          std1=np.sqrt(sum([a*b for a,b in zip(us1,us1)]))
          std2=np.sqrt(sum([a*b for a,b in zip(us2,us2)]))

          sim=(num/(std1*std2))+1

        else :sim=0

        Plist.append(sim)
      return Plist

 # #Rank based Selection
def selection(Evaluations,SortedEvaluations,pop):
    Evaluations.sort()
    Evaluations

    #Η μέγιστη αξιολόγηση θα είναι όσο και το μέγεθος του πληθυσμού(20 ή 200) και η μικρότερη θα είναι 1
    N=len(Evaluations)
    TotalFitness=(N+1)*N/2

    #Λίστα των individuals, των πιθανοτήτων, των αθροιστικών πιθανοτήτων,επιλεγμένων ατόμων,ψευδοτυχαίων αριθμών
    IndListIndexes,ProbSelection,CumProb,newList,randomy=[],[],[],[],[]
    
    #Δημιουργία ψευδτορυχαίων αριθμών και ανάθεσή τους στη λίστα
    for j in range(N):
        x = random.random()
        randomy.append(x)

    #Ανάθεση πιθανοτήτων και αθροιστικών πιθανοτήτων,  από τον χειρότερο στον καλύτερο
    for i in range(N):
        if i==0: #Εάν είναι ο πρώτος
          ProbSelection.append(1/TotalFitness)
          CumProb.append(ProbSelection[i])
        else:  #αλλιώς αθροιστικά
          ProbSelection.append((i+1)/TotalFitness)
          CumProb.append(CumProb[i-1]+ProbSelection[i])
          
    #Επιλογή
    for j in range(N):
      if randomy[j]<CumProb[0]:
          IndListIndexes.append(0)
      else:
        for e in range(N-1):
            if CumProb[e]<randomy[j] and randomy[j]<=CumProb[e+1] :
              #indexes of selected individuals to be accessed in dataframe 
                IndListIndexes.append(e+1)

    newList=SortedEvaluations.iloc[IndListIndexes].index.to_list()
    NewPop=pd.DataFrame(pop.iloc[newList])
    return NewPop

# Ομοιόμορφη διασταύρωση
# Αρχικοποιώ μία δυαδική λίστα και τη συμπληρωματική της
# Καλώ τους γονείς και αν το αντίστοιχο στοιχείο της πρώτης λίστας είναι 0 τότε ο πρώτος από τους δύο απογόνους κληρονομεί την αξιολόγηση από τον πρώτο γονέα, 
# ενώ αντίστοιχα ο άλλος απόγονος, κληρονομεί την αξιολόγηση από τον δεύτερο γονέα
def uniformX(Selected,Pc,pop_size):
    offspring1,offspring2,offsprings,generator=[],[],[],[]

    #γεννητρια ψευδοτυχαίων αριθμών προκειμένου να αποφανθούμε αν θα γίνει διαστάυρωση
    for g in range(pop_size):
        y = random.random()
        generator.append(y)

    for i in range(0,len(Selected),2):
        offspring1=[]
        offspring2=[]
        # parents
        p1=Selected.iloc[i].to_list()
        p2=Selected.iloc[i+1].to_list()
        p = [p1,p2]    

        crandomy,randomy=[],[]
        
         # Αρχικοποίηση δυαδικής λίστας
        for j in range(len(p1)):
            x = random.randint(0,1)
            randomy.append(x)
        # παίρνω το συμπλήρωμα ως προς ένα
            crandomy.append(np.abs(randomy[j]-1))          

        # Δημιουργώ τον πρώτο απόγονο διαλέγοντας για 0 την αντίστοιχη αξιολόγηση του εκάστοτε 1ου γονέα και για 1 του εκάστοτε 2ου
        # ενώ ο δεύτερος απόγονος δημιουργείται με βάση τη συμπληρωματική λίστα, με τον ίδιο τρόπο.

        if generator[i]<Pc:
          for i in range(len(p[0])):
              offspring1.append(p[randomy[i]][i])
              offspring2.append(p[crandomy[i]][i])
        else:
          for i in range(len(p[0])):
            offspring1.append(p1[i])
            offspring2.append(p2[i])
        offsprings.append(offspring1)
        offsprings.append(offspring2)
    NextGen=pd.DataFrame(offsprings)
    NextGen.index=NextGen.index+1
    NextGen.columns=columns
    NextGen.index.name = 'IndividualID'
    return NextGen

# Μετάλλαξη της αντίστοιχης αξιολόγησης, σε κάποια άλλη νόμιμη που ανήκει στο σύνολο [1,5] - {τωρινή αξιολόγηση}
def mutation(NextGeneration, Pm):
    
    # Δημιουργία array τυχαίων τιμών, διαστάσεων όσο και ο πληθυσμός
    mutArr= np.random.random(size=(NextGeneration.shape))
    # Δείκτες των θέσεων όπου η τυχαία τιμή είναι μικρότερη της πιθανότητας μετάλλαξης. 
    a=np.nonzero(mutArr <0.01)

    # Επιλογή των τιμών στις θέσεις όπου και θα πραγματοποιηθεί η μετάλλαξη
    NG1=NextGeneration.values
    g1=list(NG1[a])

    # Δημιουργία των νέων αξιολογήσεων, μέσω μετάλλαξης
    for x in range(len(g1)):
      b=random.randint(1,5)
      while True:
          if b!=g1[x]:
            break
          else:
            b=random.randint(1,5)
      g1[x]=b

    # Αντικατάσταση των νέων στοιχείων στις κατάλληλες θέσεις
    NG1[a]=g1

    # return population
    NewGeneration=pd.DataFrame(NG1)
    NewGeneration.index=NewGeneration.index+1
    NewGeneration.index.name = 'NewGenID'
    NewGeneration.columns=columns 

    return NewGeneration

def rootedMSE(a,u,new3,NewGeneration,SortedEvaluations):
    x=NewGeneration.iloc[SortedEvaluations.iloc[-1,1]-1][a].values
    y=new3.iloc[u].dropna().values
    return math.sqrt(np.mean((x-y)**2))

def meanAE(a,u,new3,NewGeneration,SortedEvaluations):
    x=NewGeneration.iloc[SortedEvaluations.iloc[-1,1]-1][a].values
    y=new3.iloc[u].dropna().values
    return np.mean(abs(x-y))

In [ ]:
# #Εύρεση χρηστών όπου οι γείτονές τους έχουν αξιολογήσει τις 10 ταινίες του συνόλου ελέγχου

usa=[]
for i in range(943):
    u1=new.iloc[i].dropna()
    a=new3.iloc[i].dropna().index
    PearList=correlate(u1,new)
    PEARSON=presentList(PearList,1)
    NeighborInd=topIndex(PEARSON)
    b=new.iloc[NeighborInd][a].any() #.values#.tolist()

    if sum(b.values)>9:
        usa.append(i)
usa[41:44]

[142, 145, 146]

In [ ]:
##########                      ΕΡΩΤΗΜΑ Β4.β                      ##############
 
#     Επιλογή index χρήστη
##    επιλογή χρήστη, 
###    αρχικοποίηση μεγέθους πληθυσμού, Pc, Pm, μέγιστου αριθμού γενεών
####   αρχικοποίηση μετρητή γενεών, μέσου αριθμού γενεών και μέσης τιμής βέλτιστου
#####  αρχικοποίηση μέσου όρου αποδόσεων για κάθε μία από τις επαναλήψεις λόγω στοχαστικής φύσης καθώς
u=154       
u1=new.iloc[u]
a=new3.iloc[u].dropna().index
pop_size,Pc,Pm,maxgen=200,0.9,0.01,300
generation,meanGen,meanBest=1,0,0
fit,temp,divide,temp2,RMSE,MAE,tempRMSE,tempMAE=[],[],[],[],[],[],[],[]
# Πλήθος επαναλήψεων λόγω στοχαστικής φύσης του αλγορίθμου, Εύρος γενεών ως όριο για τα κριτήρια τερματισμού
times=10
limit=30
 
# Συσχέτιση  user[1] με τους υπόλοιπους στο datset
PearList=correlate(u1,new)
 
# Παρουσίαση λίστας τοπ10 γειτόνων
PEARSON=presentList(PearList,1)
 
# indexes τοπ10 γειτόνων
NeighborInd=topIndex(PEARSON)
 
# find movie indexes that will be recommended
recommendations=findRecommendations2(new,new2,NeighborInd,u)
# recommendations=findRecommendations(u,new,NeighborInd)
movies=recommendations.to_list()
 
 
for i in range(times):
        # Δημιουργία πληθυσμού
        pop=[np.random.randint(1,6,len(recommendations)) for i in range(pop_size)]
        pop=pd.DataFrame(pop)
        pop.index=pop.index+1
        pop.columns=movies
        pop.index.name = 'IndividualID'
        Neighbors=pd.DataFrame(new.iloc[NeighborInd])
 
        #       Αξιολόγηση των ατόμων του πληθυσμού, με βάση τις ταινίες που έχουν δει οι γείτονες και όχι ο χρήστης
        ##      Σορτάρισμα κατά αύξουσα σειρά
        ###     Επιλογή των ατόμων που θα διασταυρωθούν
        ####    Επιλογή συγκεκριμένων ταινιών, αυτών που έχει αξιολογήσει η γειτονιά αλλά όχι ο χρήστης, και που θα αποτελέσουν τα γονίδια
        #####   Κρατάω τα indexes των ταινιών για τη δημιουργία των επόμενων dataframes
        ######  Ομοιόμορφη διασταύρωση
        ####### Μετάλλαξη
        Evaluations=fitness(pop,Neighbors,recommendations) 
        SortedEvaluations=presentList(Evaluations,0)
        NewPopulation=selection(Evaluations,SortedEvaluations,pop)
        Selected=NewPopulation.loc[:, movies]
        columns=Selected.columns.values
        NextGeneration=uniformX(Selected,Pc,pop_size)
        NewGeneration=mutation(NextGeneration,Pm)
 
        while generation<maxgen :
              generation+=1
 
              Evaluations=fitness(NewGeneration,Neighbors,recommendations) 
              SortedEvaluations=presentList(Evaluations,0)
              NewPopulation=selection(Evaluations,SortedEvaluations,NewGeneration)
 
              Selected=NewPopulation.loc[:, movies]
              columns=Selected.columns.values
                  
              NextGeneration=uniformX(Selected,Pc,pop_size)
              NewGeneration=mutation(NextGeneration,Pm)
              Evaluations=fitness(NewGeneration,Neighbors,recommendations) 
              SortedEvaluations=presentList(Evaluations,0)
              
              temp.append(SortedEvaluations.iloc[-1,0])
              print(generation,SortedEvaluations.iloc[-1,0])
              
              rmse=rootedMSE(a,u,new3,NewGeneration,SortedEvaluations)
              mae=meanAE(a,u,new3,NewGeneration,SortedEvaluations)
 
              print('rmse:',rmse,'mae:',mae)
              tempRMSE.append(rmse)
              tempMAE.append(mae)
 
####  Κριτήρια Τερματισμού  σε σχέση με το αν βελτιώνεται το καλύτερο άτομο σε βάθος γενεών ή αν βελτιώνεται κάτω από 1%
              if len(temp)>limit :
                if (SortedEvaluations.iloc[-1,0]<temp[-limit]) | ((100*(SortedEvaluations.iloc[-1,0]-temp[-limit])/temp[-limit]) <1e-1):
                    break
        temp2 = np.ones((generation-1), dtype = int)            
        meanGen+=generation
        print('temp,generation,temp2',temp,generation,temp2)
        if i==0:
            fit=temp
            divide=temp2
            RMSE=tempRMSE
            MAE=tempMAE
 
        else:
            fit=[x+y for x,y in zip_longest(fit, temp, fillvalue=0)]
            divide=[x+y for x,y in zip_longest(divide, temp2, fillvalue=0)]
            RMSE=[x+y for x,y in zip_longest(RMSE, tempRMSE, fillvalue=0)]  
            MAE=[x+y for x,y in zip_longest(MAE, tempMAE, fillvalue=0)]
        print('qumulative sum of fitnesses:',fit,'\nqumulative counter of generations:',divide,'\nqumulative RMSE:',RMSE,'\nqumulativeMAE:',MAE)
        temp.sort()
        meanBest+=temp[-1]
        print('sortedtemp,qumulbest,qumulgen:',temp,meanBest,meanGen)
        temp,temp2,tempRMSE,tempMAE=[],[],[],[]
        generation=1
 
#Μέση Τιμή Βέλτιστου
meanBest=meanBest/times
#Μέσος Αριθμός Γενεών
meanGen=meanGen/times
fit=[a/b for a,b in zip(fit,divide)]
meanRMSE=[a/b for a,b in zip(RMSE,divide)]
meanMAE=[a/b for a,b in zip(MAE,divide)]
print('fit,meanGen,meanBest',fit,meanGen,meanBest)
plt.plot(fit)
plt.ylabel('Mean Correlation')
plt.xlabel('Generations')
plt.show()
plt.plot(meanRMSE)
plt.ylabel('Mean RMSE')
plt.xlabel('Generations')
plt.show()
plt.plot(meanMAE)
plt.ylabel('Mean MAE')
plt.xlabel('Generations')
plt.show()

2 1.1357053533020827
rmse: 1.70293863659264 mae: 1.3
3 1.1248193947176899
rmse: 1.6733200530681511 mae: 1.2
4 1.1331112616690404
rmse: 1.449137674618944 mae: 1.1
